In [2]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import google.generativeai as genai
import time

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

In [ ]:
test_df = pd.read_csv("test_dataset.csv")

In [5]:
def predict_intent(text):
    prompt = f"""
Aşağıdaki cümleyi incele ve en uygun intent etiketini sadece tek kelime olarak ver:
-selamlama
-veda
-ana_ogun
-ara_ogun
-tatli
-kahvalti
-malzeme_sorgu
-diyet_onerisi
-konu_disi

Cümle: "{text}"
Intent:
"""
    try:
        response = model.generate_content(prompt)
        return response.text.strip().lower()
    except Exception as e:
        print("Hata:", e)
        return "bilinmiyor"

In [6]:
# Gerçek ve tahmin edilen etiketleri karşılaştırma
y_true = test_df["intent"].tolist()
y_pred = []

for i, row in test_df.iterrows():
    intent = predict_intent(row["text"])
    y_pred.append(intent)
    time.sleep(6)  # Gemini hız limiti için bekleme

# Performans metriklerini yazdır
print("Gemini Classification Report:")
print(classification_report(y_true, y_pred))

print("Gemini Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Gemini Classification Report:
               precision    recall  f1-score   support

     ana_ogun       1.00      0.97      0.98        30
     ara_ogun       0.97      1.00      0.98        30
diyet_onerisi       1.00      0.97      0.98        30
     kahvalti       0.97      1.00      0.98        30
    konu_disi       1.00      0.90      0.95        30
malzeme_sorgu       1.00      1.00      1.00        30
    selamlama       0.86      1.00      0.92        30
        tatli       1.00      1.00      1.00        30
         veda       1.00      0.93      0.97        30

     accuracy                           0.97       270
    macro avg       0.98      0.97      0.97       270
 weighted avg       0.98      0.97      0.97       270

Gemini Confusion Matrix:
[[29  0  0  1  0  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0]
 [ 0  1 29  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0]
 [ 0  0  0  0 27  0  3  0  0]
 [ 0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0 30  0  0]
 [ 0  0  0  0  0 